In [17]:
import os
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Veri seti yolu
data_path = "/content/drive/MyDrive/gdrive/Fire-Detection"

# Sınıf etiketleri
class_names = ["0", "1"]

# Veri setindeki görüntü sayısı
num_samples = sum([len(files) for r, d, files in os.walk(data_path)])

# Veri seti boyutları
img_height, img_width = 128, 128
num_channels = 3  # Görüntülerin RGB kanal sayısı (3 kanal)

# X ve y için numpy dizileri
X = np.zeros((num_samples, img_height, img_width, num_channels), dtype=np.float32)
y = np.zeros((num_samples,), dtype=np.int32)

# Verileri yükleyip numpy dizilerine dönüştürme
index = 0
for i, class_name in enumerate(class_names):
    class_path = os.path.join(data_path, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path)
        img = img.convert("RGB")  # Görüntüyü RGB moduna dönüştürmek
        img = img.resize((img_height, img_width))
        X[index] = np.array(img, dtype=np.float32) / 255.
        y[index] = i
        index += 1

# Veri setini karıştırma
np.random.seed(42)
shuffle_index = np.random.permutation(num_samples)
X, y = X[shuffle_index], y[shuffle_index]

# Veri setini train, test ve validation olarak ayırma
num_train_samples = int(0.7 * num_samples)
num_test_samples = int(0.2 * num_samples)
num_val_samples = num_samples - num_train_samples - num_test_samples

x_train, y_train = X[:num_train_samples], y[:num_train_samples]
x_test, y_test = X[num_train_samples:num_train_samples + num_test_samples], y[
                                                                            num_train_samples:num_train_samples + num_test_samples]
x_val, y_val = X[num_train_samples + num_test_samples:], y[num_train_samples + num_test_samples:]

# Dizileri npy dosyalarına kaydetme
np.save('x_train.npy', x_train)
np.save('y_train.npy', y_train)
np.save('x_test.npy', x_test)
np.save('y_test.npy', y_test)
np.save('x_val.npy', x_val)
np.save('y_val.npy', y_val)

# Model oluşturma
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, num_channels)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Modeli derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Test verisi üzerinde modeli değerlendirme
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/10
15/15 [==============================] - 2s 38ms/step - loss: 0.3687 - accuracy: 0.8374 - val_loss: 0.4859 - val_accuracy: 0.8030
Epoch 2/10
15/15 [==============================] - 0s 34ms/step - loss: 0.3108 - accuracy: 0.8703 - val_loss: 0.3495 - val_accuracy: 0.8485
Epoch 3/10
15/15 [==============================] - 0s 20ms/step - loss: 0.2586 - accuracy: 0.8901 - val_loss: 0.3417 - val_accuracy: 0.8636
Epoch 4/10
15/15 [==============================] - 0s 33ms/step - loss: 0.2349 - accuracy: 0.8923 - val_loss: 0.3925 - val_accuracy: 0.7879
Epoch 5/10
15/15 [==============================] - 0s 20ms/step - loss: 0.2929 - accuracy: 0.8923 - val_loss: 0.3108 - val_accuracy: 0.8636
Epoch 6/10
15/15 [==============================] - 0s 27ms/step - loss: 0.2207 - accuracy: 0.9033 - val_loss: 0.2770 - val_accuracy: 0.9091
Epoch 7/10
15/15 [==============================] - 0s 25ms/step - loss: 0.1728 - accuracy: 0.9187 - val_loss: 0.3664 - val_accuracy: 0.8788
Epoch 8/10
15

In [26]:
# Yeni görüntünün yolu
new_image_path = "/content/ov7670.jpeg"

# Görüntüyü yükleme ve işleme
new_image = Image.open(new_image_path)
new_image = new_image.resize((img_height, img_width))
new_image_array = np.array(new_image, dtype=np.float32) / 255.
new_image_array = new_image_array[:, :, :3]  # Yalnızca ilk üç kanalı al

# Tahmin yapma
prediction = model.predict(np.expand_dims(new_image_array, axis=0))

# Tahmin sonucunu yazdırma
if prediction > 0.5:
    print("Görüntü yangın içeriyor.")
else:
    print("Görüntüde yangın yok.")


1/1 [==============================] - 0s 18ms/step
Görüntü yangın içeriyor.
